In [18]:
import pandas as pd
import pickle
import seaborn as sns

# Necessary imports
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

In [1]:
def diagnostic_plot(x, y):
    plt.figure(figsize=(20,5))
    
    rgr = LinearRegression()
    rgr.fit(x,y)
    pred = rgr.predict(x)

    plt.subplot(1, 3, 1)
    plt.scatter(x,y)
    plt.plot(x, pred, color='blue',linewidth=1)
    plt.title("Regression fit")
    plt.xlabel("x")
    plt.ylabel("y")
    
    plt.subplot(1, 3, 2)
    res = y - pred
    plt.scatter(pred, res)
    plt.title("Residual plot")
    plt.xlabel("prediction")
    plt.ylabel("residuals")
    
    plt.subplot(1, 3, 3)
    #Generates a probability plot of sample data against the quantiles of a 
    # specified theoretical distribution 
    stats.probplot(res, dist="norm", plot=plt)
    plt.title("Normal Q-Q plot")

In [2]:
new_era = pickle.load(open('ready_for_model_new_era.p','rb'))
new_era['new_era'] = 1

NameError: name 'pickle' is not defined

In [3]:
new_era.sample(5)

NameError: name 'new_era' is not defined

In [4]:
old_era = pickle.load(open('ready_for_model_old_era.p','rb'))
old_era['new_era'] = 0

NameError: name 'pickle' is not defined

In [5]:
old_era.sample(5)

NameError: name 'old_era' is not defined

In [6]:
combined_data = pd.concat([new_era,old_era])

NameError: name 'pd' is not defined

In [7]:
combined_data['win_per']=combined_data['win_per']*100

NameError: name 'combined_data' is not defined

In [8]:
sns.pairplot(combined_data.loc[:,['ORtg_p1', 'ORtg_p2','ORtg_p3','ORtg_p4','ORtg_p5','win_per']]);

NameError: name 'sns' is not defined

In [9]:
sns.pairplot(combined_data.loc[:,['DRtg_p1', 'DRtg_p2','DRtg_p3','DRtg_p4','DRtg_p5','win_per']]);

NameError: name 'sns' is not defined

In [10]:
# Create your feature matrix (X) and target vector (y)
y, X = patsy.dmatrices('win_per ~ ORtg_p1 + ORtg_p2+ ORtg_p3+ ORtg_p4+ ORtg_p5 + DRtg_p1 + DRtg_p2+ DRtg_p3+ DRtg_p4+ DRtg_p5 + new_era', data=combined_data, return_type="dataframe")



# Create your model
model = sm.OLS(y, X)

# Fit your model to your training set
fit = model.fit()

# Print summary statistics of the model's performance
fit.summary()

NameError: name 'patsy' is not defined

In [11]:
pred = fit.predict(X)
res = y.iloc[:,0] - pred
plt.scatter(pred, res)
plt.title("Residual plot")
plt.xlabel("prediction")
plt.ylabel("residuals");
    


NameError: name 'fit' is not defined

In [12]:
stats.probplot(res, dist="norm", plot=plt)
plt.title("Normal Q-Q plot");

NameError: name 'stats' is not defined

In [13]:
plt.scatter(y=res,x=combined_data['year']);

NameError: name 'plt' is not defined

In [14]:
sns.violinplot(y=res,x=combined_data['year']);

NameError: name 'sns' is not defined

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures



In [16]:

X, y = combined_data.drop('win_per',axis=1), combined_data['win_per']

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=11) #hold out 20% of the data for final testing

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

NameError: name 'combined_data' is not defined

In [ ]:
#run the CV
mean_r2s =[]
std_r2s =[]
lambs =[]
for n in  np.linspace(25,75,51):

    kf = KFold(n_splits=5, shuffle=True, random_state = 71)
    cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models
    cv_lm_poly_r2s=[]
    for train_ind, val_ind in kf.split(X,y):

        X_train, y_train = X[train_ind], y[train_ind]
        X_val, y_val = X[val_ind], y[val_ind] 

        #simple linear regression
        lm = LinearRegression()
        lm_reg = Ridge(alpha=n)

        lm.fit(X_train, y_train)
        cv_lm_r2s.append(lm.score(X_val, y_val))

        #ridge with feature scaling
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        lm_reg.fit(X_train_scaled, y_train)
        cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

        #polynomial regression
        poly = PolynomialFeatures(degree=2)

        X_train_poly = poly.fit_transform(X_train)
        X_val_poly = poly.transform(X_val)
        X_test_poly = poly.transform(X_test)

        lm_poly = LinearRegression()
        lm_poly.fit(X_train_poly, y_train)
        cv_lm_poly_r2s.append(lm_poly.score(X_val_poly, y_val))
        
        mean_r2s.append(np.mean(cv_lm_reg_r2s))
        std_r2s.append(np.std(cv_lm_reg_r2s))
        lambs.append(n)

    #

# print('Simple regression scores: ', cv_lm_r2s)
# print('Ridge scores: ', cv_lm_reg_r2s)
# print('Poly scores: ', cv_lm_poly_r2s,'\n')

# print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
# print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}' ,)
# print(f'Poly mean cv r^2: {np.mean(cv_lm_poly_r2s):.3f} +- {np.std(cv_lm_poly_r2s):.3f}' ,)




In [17]:
plt.plot(lambs,mean_r2s);

NameError: name 'plt' is not defined